# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
Arxiv has 95 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.15267


extracting tarball to tmp_2310.15267...

 done.
Retrieving document from  https://arxiv.org/e-print/2310.15879


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2310.15267/aassymbols.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2310.16038/lem_wp.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'bib' from 'tmp_2310.16038/bib.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'headstuff' from 'tmp_2310.16038/headstuff.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.

/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:730: LatexWarning: Could not extract abstract from tmp_2310.16038/lem_wp.tex
  warnings.warn(LatexWarning(f"Could not extract abstract from {self.main_file}"))


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure fig/lem_spec_whim_r1
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure fig/lem_spec_whim_r1 as tmp_2310.16038/./fig/lem_spec_whim_r1.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


/tmp/ipykernel_2262/3009462391.py:49: LatexWarning: 2310.16038 did not run properly
Unable to locate Ghostscript on paths
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.15895-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.15895) | **A roadmap to the efficient and robust characterization of temperate  terrestrial planet atmospheres with JWST**  |
|| J. d. Wit, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2023-10-25*|
|*Comments*| **|
|**Abstract**| Ultra-cool dwarf stars are abundant, long-lived, and uniquely suited to enable the atmospheric study of transiting terrestrial companions with JWST. Amongst them, the most prominent is the M8.5V star TRAPPIST-1 and its seven planets, which have been the favored targets of eight JWST Cycle 1 programs. While Cycle 1 observations have started to yield preliminary insights into the planets, they have also revealed that their atmospheric exploration requires a better understanding of their host star. Here, we propose a roadmap to characterize the TRAPPIST-1 system -- and others like it -- in an efficient and robust manner. We notably recommend that -- although more challenging to schedule -- multi-transit windows be prioritized to constrain stellar heterogeneities and gather up to 2$\times$ more transits per JWST hour spent. We conclude that in such systems planets cannot be studied in isolation by small programs, thus large-scale community-supported programs should be supported to enable the efficient and robust exploration of terrestrial exoplanets in the JWST era. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.15267-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.15267) | **Heat-Flux Limited Cloud Activity and Vertical Mixing in Giant Planet  Atmospheres with an Application to Uranus and Neptune**  |
|| H. Ge, C. Li, <mark>X. Zhang</mark>, C. Moeckel |
|*Appeared on*| *2023-10-25*|
|*Comments*| *23 pages, 7 figures, and 2 tables. Accepted for publication in PSJ*|
|**Abstract**| Storms operated by moist convection and the condensation of $\rm CH_{4}$ or $\rm H_{2}S$ have been observed on Uranus and Neptune. However, the mechanism of cloud formation, thermal structure, and mixing efficiency of ice giant weather layers remains unclear. In this paper, we show that moist convection is limited by heat transport on giant planets, especially on ice giants where planetary heat flux is weak. Latent heat associated with condensation and evaporation can efficiently bring heat across the weather layer through precipitations. This effect was usually neglected in previous studies without a complete hydrological cycle. We first derive analytical theories and show the upper limit of cloud density is determined by the planetary heat flux and microphysics of clouds but independent of the atmospheric composition. The eddy diffusivity of moisture depends on the heat fluxes, atmospheric composition, and gravity of the planet but is not directly related to cloud microphysics. We then conduct convection- and cloud-resolving simulations with SNAP to validate our analytical theory. The simulated cloud density and eddy diffusivity are smaller than the results acquired from the equilibrium cloud condensation model and mixing length theory by several orders of magnitude but consistent with our analytical solutions. Meanwhile, the mass-loading effect of $\rm CH_{4}$ and $\rm H_{2}S$ leads to superadiabatic and stable weather layers. Our simulations produced three cloud layers that are qualitatively similar to recent observations. This study has important implications for cloud formation and eddy mixing in giant planet atmospheres in general and observations for future space missions and ground-based telescopes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.15879-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.15879) | **Prospects for probing the interaction between dark energy and dark  matter using gravitational-wave dark sirens with neutron star tidal  deformation**  |
|| T.-N. Li, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-10-25*|
|*Comments*| *12 pages, 9 figures*|
|**Abstract**| Gravitational wave (GW) standard siren observations provide a rather useful tool to explore the evolution of the universe. In this work, we wish to investigate whether the dark sirens with neutron star (NS) deformation from third-generation (3G) GW detectors could help probe the interaction between dark energy and dark matter. We simulate the GW dark sirens of four detection strategies based on the three-year observation and consider four phenomenological interacting dark energy models to perform cosmological analysis. We find that GW dark sirens could provide tight constraints on $\Omega_{\rm m}$ and $H_0$ in the four IDE models, but perform not well in constraining the dimensionless coupling parameter $\beta$ with the interaction proportional to the energy density of cold dark matter. Nevertheless, the parameter degeneracy orientations of CMB and GW are almost orthogonal, and thus the combination of them could effectively break cosmological parameter degeneracies, with the constraint errors of $\beta$ being 0.00068-0.018. In addition, we choose three typical equation of states (EoSs) of NS, i.e., SLy, MPA1, and MS1, to investigate the effect of NS's EoS in cosmological analysis. The stiffer EoS could give tighter constraints than the softer EoS. Nonetheless, the combination of CMB and GW dark sirens (using different EoSs of NS) shows basically the same constraint results of cosmological parameters. We conclude that the dark sirens from 3G GW detectors would play a crucial role in helping probe the interaction between dark energy and dark matter, and the CMB+GW results are basically not affected by the EoS of NS. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.15925-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.15925) | **Noema formIng Cluster survEy (NICE): Discovery of a starbursting galaxy  group with a radio-luminous core at z=3.95**  |
|| L. Zhou, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2023-10-25*|
|*Comments*| *7 pages, 7 figures, submitted to A&A*|
|**Abstract**| The study of distant galaxy groups and clusters at the peak epoch of star formation is limited by the lack of a statistically and homogeneously selected and spectroscopically confirmed sample. Recent discoveries of concentrated starburst activities in cluster cores have opened a new window to hunt for these structures based on their integrated IR luminosities. Hereby we carry out the large NOEMA (NOrthern Extended Millimeter Array) program targeting a statistical sample of infrared-luminous sources associated with overdensities of massive galaxies at z>2, the Noema formIng Cluster survEy (NICE). We present the first result from the ongoing NICE survey, a compact group at z=3.95 in the Lockman Hole field (LH-SBC3), confirmed via four massive (M_star>10^10.5M_sun) galaxies detected in CO(4-3) and [CI](1-0) lines. The four CO-detected members of LH-SBC3 are distributed over a 180 kpc physical scale, and the entire structure has an estimated halo mass of ~10^13Msun and total star formation rate (SFR) of ~4000Msun/yr. In addition, the most massive galaxy hosts a radio-loud AGN with L_1.4GHz, rest = 3.0*10^25W/Hz. The discovery of LH-SBC3 demonstrates the feasibility of our method to efficiently identify high-z compact groups or forming cluster cores. The existence of these starbursting cluster cores up to z~4 provides critical insights into the mass assembly history of the central massive galaxies in clusters. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.16038-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.16038) | **LEM All-Sky Survey: Soft X-ray Sky at Microcalorimeter Resolution**  |
|| I. Khabibullin, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2023-10-25*|
|*Comments*| *White Paper in support of a mission concept to be submitted for the 2023 NASA Astrophysics Probes opportunity. This White Paper will be updated when required. 30 pages, 25 figures*|
|**Abstract**| The Line Emission Mapper (LEM) is an X-ray Probe with with spectral resolution ~2 eV FWHM from 0.2 to 2.5 keV and effective area >2,500 cm$^2$ at 1 keV, covering a 33 arcmin diameter Field of View with 15 arcsec angular resolution, capable of performing efficient scanning observations of very large sky areas and enabling the first high spectral resolution survey of the full sky. The LEM-All-Sky Survey (LASS) is expected to follow the success of previous all sky surveys such as ROSAT and eROSITA, adding a third dimension provided by the high resolution microcalorimeter spectrometer, with each 15 arcsec pixel of the survey including a full 1-2 eV resolution energy spectrum that can be integrated over any area of the sky to provide statistical accuracy. Like its predecessors, LASS will provide both a long-lasting legacy and open the door to the unknown, enabling new discoveries and delivering the baseline for unique GO studies. No other current or planned mission has the combination of microcalorimeter energy resolution and large grasp to cover the whole sky while maintaining good angular resolution and imaging capabilities. LASS will be able to probe the physical conditions of the hot phases of the Milky Way at multiple scales, from emission in the Solar system due to Solar Wind Charge eXchange, to the interstellar and circumgalactic media, including the North Polar Spur and the Fermi/eROSITA bubbles. It will measure velocities of gas in the inner part of the Galaxy and extract the emissivity of the Local Hot Bubble. By maintaining the original angular resolution, LASS will also be able to study classes of point sources through stacking. For classes with ~$10^4$ objects, it will provide the equivalent of 1 Ms of high spectral resolution data. We describe the technical specifications of LASS and highlight the main scientific objectives that will be addressed. (Abridged) |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Unable to locate Ghostscript on paths</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2310.15895.md


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

, and marker sizes are proportional to planet sizes.
The region of temperature planets with instellations between 0.25 and 4 $S_\oplus$ is highlighted, and planets of particular interest are labelled.
 (*fig:figure1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.15895"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

307  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
